In [3]:
import numpy as np
from random import shuffle

def svm_loss_naive(W, X, y, reg):
    d, C = W.shape
    _, N = X.shape
    
    
    loss = 0 
    dW = np.zeros_like(W)
    
    for n in range(N):
        xn = X[:, n]
        score = W.T.dot(xn)
        for j in range(C):
            if j == y[n]:
                continue
            margin = 1 - score[y[n]] + score[j]
            if margin > 0:
                loss+=margin
                dW[:, j] += xn
                dW[:, y[n]] -= xn
        
    loss /= N
    loss += 0.5*reg*np.sum(W*W)
    
    dW /= N
    dW += reg*W
    
    return loss, dW
N, C, d = 10, 3, 5

reg = .1
W = np.random.randn(d, C)
X = np.random.randn(d, N)
y = np.random.randint(C, size=N)

print('loss without regularization :%f ' %svm_loss_naive(W, X, y, 0)[0])
print('loss without regularization :%f ' %svm_loss_naive(W, X, y, .1)[0])

loss without regularization :1.812326 
loss without regularization :2.391587 


In [6]:
f = lambda W: svm_loss_naive(W, X, y, .1)[0]

# for checking if calculated grad is correct
def numerical_grad_general(W, f):
    eps = 1e-6
    g = np.zeros_like(W)
    # flatening variable -> 1d. Then we need 
    # only one for loop
    W_flattened = W.flatten()
    g_flattened = np.zeros_like(W_flattened)
    
    for i in range(W.size):
        W_p = W_flattened.copy()
        W_n = W_flattened.copy()
        W_p[i] += eps 
        W_n[i] -= eps 
        
        # back to shape of W 
        W_p = W_p.reshape(W.shape)
        W_n = W_n.reshape(W.shape)
        g_flattened[i] = (f(W_p) - f(W_n))/(2*eps)
        
    # convert back to original shape
    return g_flattened.reshape(W.shape) 

# compare two ways of computing gradient
g1 = svm_loss_naive(W, X, y, .1)[1]
g2 = numerical_grad_general(W, f)
print ('gradient difference: %f' %np.linalg.norm(g1 - g2))
# this should be very small


gradient difference: 0.000000


In [7]:
def svm_loss_vectorized(W, X, y, reg):
    d, C = W.shape
    _, N = X.shape 
    loss = 0
    dW = np.zeros_like(W)
    
    Z = W.T.dot(X)
    correct_class_score = np.choose(y, X).reshape(N, 1).T
    margins = np.maximum(0, Z - correct_class_score + 1)
    margins[y, np.arange(margins.shape[1])] = 0
    loss = np.sum(margins, axis=(0, 1))
    loss /= N 
    loss += 0.5 * reg * np.sum(W * W)
    
    F = (margins > 0).astype(int)
    F[y, np.arange(F.shape[1])] = np.sum(-F, axis = 0)
    dW = X.dot(F.T)/N + reg*W
    return loss, dW

In [10]:
N, C, d = 49000, 10, 3073
reg = .1 
W = np.random.randn(d, C)
X = np.random.randn(d, N)
y = np.random.randint(C, size = N)

import time 
t1 = time.time()
l1, dW1 = svm_loss_naive(W, X, y, reg)
t2 = time.time()
print('Naive     : run time:', t2 - t1, '(s)')

t1 = time.time()
l2, dW2 = svm_loss_vectorized(W, X, y, reg)
t2 = time.time()
print('Vectorized: run time:', t2 - t1, '(s)')
print('loss difference:', np.linalg.norm(l1 - l2))
print('gradient difference:', np.linalg.norm(dW1 - dW2))


MemoryError: Unable to allocate array with shape (3073, 49000) and data type float64